In [1]:
import requests
import pandas as pd
import uuid

# SQLITE3 to manage a very simple database
# https://www.sqlite.org/
import sqlite3

In [2]:
api_key = "f275aa333c628e5a702c7d4f1fce6cdb"

cities_dict = {
    0: ["Mieres", "ES"],
    1: ["Sevilla", "ES"],
    2: ["Zarautz", "ES"],
    3: ["Barcelona", "ES"],
    4: ["Madrid", "ES"]
}

In [4]:
# Create if not exists
# or connect to the SQLITE3 database
conn = sqlite3.connect('db.sqlite3')

# Generate a cursor to access to the database
c = conn.cursor()

In [5]:
create_cities_query = """
    CREATE TABLE IF NOT EXISTS city (
        [uuid] TEXT PRIMARY KEY, 
        [name] TEXT,
        [country] TEXT,
        [lon] TEXT, 
        [lat] TEXT        
    );
"""

create_measures_query = """
    CREATE TABLE IF NOT EXISTS measure (
        [uuid] TEXT PRIMARY KEY, 
        [timestamp] TEXT,
        [city_uuid] TEXT,
        [humi] INTEGER,
        [temp] REAL,
        [pressure] INTEGER,
        [wind] REAL,
        FOREIGN KEY(city_uuid) REFERENCES city(uuid)
    );
"""

select_all_cities = """
    SELECT
        uuid,
        name,
        country,
        lon, 
        lat
    FROM city;
"""

select_all_measures = """
    SELECT
        uuid,
        timestamp,
        city_uuid,
        humi, 
        temp,
        pressure,
        wind
    FROM measure;
"""


In [6]:
c.execute(create_cities_query)
c.execute(create_measures_query)
                     
conn.commit()

In [7]:
c.execute(select_all_cities)

In [8]:
all_cities = c.fetchall()

In [9]:
cities_df = pd.read_sql_query("SELECT * FROM city", conn)

In [25]:
try:
    print(cities_df.set_index("uuid").head(10))
except Exception as e:
    pass

                                           name country      lon      lat
uuid                                                                     
93fbb7ec-74e9-405f-a4f5-f01611d92416     Mieres      ES  -5.7667    43.25
2a7e7843-f6f4-4fa7-aeb8-1086ce96b2ee    Sevilla      ES  -5.9761  37.3824
69b6e1cc-853a-44ae-b699-36184a29a159    Zarautz      ES  -2.1699  43.2844
a9b7056b-cab4-4ebb-9a1d-905fa692d866  Barcelona      ES    2.159  41.3888
25c121b7-69ba-4ebc-bfc0-cf19ef9ca4ae     Madrid      ES  -3.7026  40.4165


In [62]:
if len(all_cities) < 1:

    coords_dict = {}

    for k, v in cities_dict.items():
        URL = "https://api.openweathermap.org/data/2.5/weather?q={0},{1}&appid={2}" \
            .format(v[0], v[1], api_key)
        response = requests.get(url = URL)
        data = response.json()
        coords_dict[k] = [
            data["coord"]["lon"], 
            data["coord"]["lat"]]

    for city, coord in zip(cities_dict.items(), coords_dict.items()):

        _uuid = str(uuid.uuid4())
        name = city[1][0]
        country = city[1][1]
        lon = coord[1][0]
        lat = coord[1][1]

        #print(_uuid, name, country, lon, lat)

        insert_into_city = """
            INSERT INTO city 
                (uuid, name, country, lon, lat)
                VALUES ('{0}', '{1}', '{2}', {3}, {4});
            """.format(_uuid, name, country, lon, lat)

        c.execute(insert_into_city)
        conn.commit()  

else:

    measures_dict = {}

    for city in all_cities:

        _uuid = str(uuid.uuid4())
        city_uuid = city[0]
        lat = city[3]
        lon = city[4]
        URL = "https://api.openweathermap.org/data/2.5/weather?lat={1}&lon={0}&appid={2}&units=metric" \
            .format(lat, lon, api_key)
        PARAMS = {}
        response = requests.get(url = URL, params = PARAMS)
        data = response.json() 
        humi = data["main"]["humidity"]
        temp = data["main"]["temp"]
        pressure = data["main"]["pressure"]
        wind = data["wind"]["speed"]
        timestamp = data["dt"]

        #print(_uuid, timestamp, city_uuid, humi, temp, pressure, wind)

        insert_into_measure = """
            INSERT INTO measure 
                (uuid, timestamp, city_uuid, humi, temp, pressure, wind)
                VALUES ('{0}', '{1}', '{2}', {3}, {4}, {5}, {6});
            """.format(_uuid, timestamp, city_uuid, humi, temp, pressure, wind) 

        c.execute(insert_into_measure)
        conn.commit()         



69d381fe-4bcc-4907-9091-7c541999c237 1654553165 93fbb7ec-74e9-405f-a4f5-f01611d92416 93 14.5 1022 0.68
65cfc118-867c-4b2a-ac12-bcf6ef8dbc37 1654552613 2a7e7843-f6f4-4fa7-aeb8-1086ce96b2ee 40 24.41 1016 2.57
819ec908-8732-447c-8935-911f363559fa 1654553126 69b6e1cc-853a-44ae-b699-36184a29a159 83 17.63 1022 1.31
9c06c647-6fcc-4ef7-9781-7c1df1cd5d20 1654552500 a9b7056b-cab4-4ebb-9a1d-905fa692d866 69 23.25 1017 5.14
aa738668-38db-43e5-99e9-e93fbd26a611 1654552693 25c121b7-69ba-4ebc-bfc0-cf19ef9ca4ae 34 24.48 1005 3.09


In [20]:
measures_df = pd.read_sql_query("SELECT * FROM measure", conn)
measures_df["datetime"] = pd.to_datetime(measures_df["timestamp"], unit = 's') 

In [21]:
measures_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   uuid       5 non-null      object        
 1   timestamp  5 non-null      object        
 2   city_uuid  5 non-null      object        
 3   humi       5 non-null      int64         
 4   temp       5 non-null      float64       
 5   pressure   5 non-null      int64         
 6   wind       5 non-null      float64       
 7   datetime   5 non-null      datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(2), object(3)
memory usage: 448.0+ bytes


In [26]:
print(measures_df.set_index("uuid").head(10))

                                       timestamp  \
uuid                                               
69d381fe-4bcc-4907-9091-7c541999c237  1654553165   
65cfc118-867c-4b2a-ac12-bcf6ef8dbc37  1654552613   
819ec908-8732-447c-8935-911f363559fa  1654553126   
9c06c647-6fcc-4ef7-9781-7c1df1cd5d20  1654552500   
aa738668-38db-43e5-99e9-e93fbd26a611  1654552693   

                                                                 city_uuid  \
uuid                                                                         
69d381fe-4bcc-4907-9091-7c541999c237  93fbb7ec-74e9-405f-a4f5-f01611d92416   
65cfc118-867c-4b2a-ac12-bcf6ef8dbc37  2a7e7843-f6f4-4fa7-aeb8-1086ce96b2ee   
819ec908-8732-447c-8935-911f363559fa  69b6e1cc-853a-44ae-b699-36184a29a159   
9c06c647-6fcc-4ef7-9781-7c1df1cd5d20  a9b7056b-cab4-4ebb-9a1d-905fa692d866   
aa738668-38db-43e5-99e9-e93fbd26a611  25c121b7-69ba-4ebc-bfc0-cf19ef9ca4ae   

                                      humi   temp  pressure  wind  \
uuid               